In [1]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import asyncio
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
open_router_key = os.getenv("OPENROUTER_API_KEY")

In [2]:
model_client = OpenAIChatCompletionClient(
    model='gemini-1.5-flash',
    api_key=api_key,
)

In [3]:
from autogen_agentchat.teams import DiGraphBuilder, GraphFlow

## Sequential Flow

In [4]:
writer = AssistantAgent(
    name="Writer",
    model_client=model_client,
    system_message="You are a helpful writer.",
    description="Writes content based on the user's input.",
)

reviewer = AssistantAgent(
    name="Reviewer",
    model_client=model_client,
    system_message="You are a helpful reviewer.",
    description="Reviews the content written by the writer.",
)

builder = DiGraphBuilder()
builder.add_node(writer)
builder.add_node(reviewer)

builder.add_edge(writer, reviewer)

graph = builder.build()

In [5]:
team = GraphFlow(participants=[writer, reviewer], graph=graph)

In [ ]:
import asyncio
async def main():
    stream = team.run_stream(task="Write a 5 line poem with rhyming words about indian culture.")
    async for event in stream:
        print(event)

asyncio.run(main())

source='user' models_usage=None metadata={} content='Write a 5 line poem with rhyming words about indian culture.' type='TextMessage'
source='Writer' models_usage=RequestUsage(prompt_tokens=19, completion_tokens=42) metadata={} content='With vibrant hues, a joyful sight,\nA land of spices, shining bright.\nFrom Ganges flows a sacred stream,\nA tapestry of ancient dream,\nIn rich traditions, hearts aflame.\n' type='TextMessage'
source='Reviewer' models_usage=RequestUsage(prompt_tokens=61, completion_tokens=111) metadata={} content='This is a lovely little poem!  The imagery is strong and evocative, successfully conveying a sense of vibrancy and tradition.  The rhyme scheme is consistent and pleasing to the ear.  My only minor suggestion would be to consider slightly less cliché imagery.  While "spices" and the "Ganges" are associated with India, they\'re very common tropes.  Perhaps exploring a less expected image could elevate the poem\'s originality.  But as it stands, it\'s a well-wr

## Parallel Flow

In [20]:
writer = AssistantAgent(
    name="Writer",
    model_client=model_client,
    system_message="You are a helpful poem writer keep the content concise and engaging and do some gramatical mistakes and not keep up with rhyming words and within 20 words max.",
    description="Writes content based on the user's input.",
)

editor1 = AssistantAgent(
    name="Editor1",
    model_client=model_client,
    system_message="You are a helpful editor for editing the gramatical mistakes in the poem and tell the changes after doing and correct them and within 20 words max.",
    description="Edits the content written by the writer.",
)

editor2 = AssistantAgent(
    name="Editor2",
    model_client=model_client,
    system_message="You are a helpful editor for editing the Rhyiming words and the styles and check content is valid in the poem and correct them tell the changes after doing and within 20 words max.",
    description="Edits the content written by the writer.",
)

reviewer = AssistantAgent(
    name="Reviewer",
    model_client=model_client,
    system_message="You are a helpful reviewer for reviewing the content written by the writer and editors and give the final output of the poem after edited and within 20 words max.",
    description="Reviews the content written by the writer and editors.",
)

In [21]:
builder = DiGraphBuilder()

builder.add_node(writer)
builder.add_node(editor1)
builder.add_node(editor2)
builder.add_node(reviewer)

builder.add_edge(writer, editor1)
builder.add_edge(writer, editor2)
builder.add_edge(editor1, reviewer)
builder.add_edge(editor2, reviewer)

graph = builder.build()

In [22]:
from autogen_agentchat.ui import Console
flow = GraphFlow(participants=[writer, editor1, editor2, reviewer], graph=graph)

await Console(flow.run_stream(task="Write a 10 line poem with rhyming words about Tamil culture."))

---------- TextMessage (user) ----------
Write a 10 line poem with rhyming words about Tamil culture.
---------- TextMessage (Writer) ----------
Tamil pride, a vibrant sight,
Temples old, shining bright.
Dance and song, a joyful sound,
Ancient stories, all around.
Spicy food, a flavour burst,
Rich traditions, never worst.
Family bonds, strong and true,
Culture's beauty, shines anew.
Generations, keep it alive,
Tamil spirit, will survive.

---------- TextMessage (Editor1) ----------
Corrected a few minor grammatical issues for smoother flow.  No major changes needed.

---------- TextMessage (Editor2) ----------
Improved flow and rhyme slightly.  Minor word changes for better rhythm.

---------- TextMessage (Reviewer) ----------
Tamil culture, vibrant and bold,
Ancient stories, ages old.
Dance and song, a joyful sound,
Rich traditions, all around.
Spicy food, a flavour bright,
Family bonds, shining light.
Generations, keep it alive,
Spirit strong, it will survive.
Temples gleam, a sacred

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a 10 line poem with rhyming words about Tamil culture.', type='TextMessage'), TextMessage(source='Writer', models_usage=RequestUsage(prompt_tokens=47, completion_tokens=79), metadata={}, content="Tamil pride, a vibrant sight,\nTemples old, shining bright.\nDance and song, a joyful sound,\nAncient stories, all around.\nSpicy food, a flavour burst,\nRich traditions, never worst.\nFamily bonds, strong and true,\nCulture's beauty, shines anew.\nGenerations, keep it alive,\nTamil spirit, will survive.\n", type='TextMessage'), TextMessage(source='Editor1', models_usage=RequestUsage(prompt_tokens=124, completion_tokens=17), metadata={}, content='Corrected a few minor grammatical issues for smoother flow.  No major changes needed.\n', type='TextMessage'), TextMessage(source='Editor2', models_usage=RequestUsage(prompt_tokens=148, completion_tokens=15), metadata={}, content='Improved flow and rhyme sli